In [203]:
# import important libraries

In [508]:
import sys
import math
import numpy as np
import ast
import itertools
import nltk
import elasticsearch
from sklearn import tree
from nltk.tree import Tree
import operator

In [509]:
# typing your question

In [ ]:
script, question = sys.argv

In [510]:
# script, question = sys.argv

In [511]:
# get our corpus

In [512]:
with open('all_news.txt','r') as f:
    corpus = ast.literal_eval(f.read())
    corpus = list(itertools.chain(*corpus))
    corpus = list(itertools.chain(*corpus))

In [513]:
# connect the elasticsearch local server and indexing the document

In [514]:
from elasticsearch import Elasticsearch
import datetime
from datetime import datetime
from elasticsearch.helpers import bulk

In [579]:
client = Elasticsearch()
documents = []

for i in range(len(corpus)):
    process_sent = corpus[i]
    
    document = {
        "_index" : "articles-index",
        "_type" : "articles",
        "_id" : i,
        "_source" : {
            "any" : "data" + str(i),
            "timestamp": datetime.now(),
            "body" : process_sent
        }
    }
    
    documents.append(document)
if len(documents) > 0:
    bulk(client, documents)
else:
    raise Exception('There is no document')


In [516]:
# Question Analysis

In [517]:
# 1.define question classifier

In [534]:
question_1 = 'which companies went bankrupt in month x of year y?'
question_2 = 'what affects gdp?'
question_3 = 'what percentage of drop or increase change in gdp is associated with z?'
question_4 = 'Who is the ceo of company x?'
questions =[question_1, question_2, question_3, question_4]

In [535]:
import re
from collections import Counter

In [536]:
WORD = re.compile(r'\w+')
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    
    sum1 = sum([vec1[x] ** 2 for x in vec1.keys()])
    sum2 = sum([vec2[x] ** 2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    
    if not denominator: 
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [537]:
vector_1 = text_to_vector(question_1)
vector_2 = text_to_vector(question_2)
vector_3 = text_to_vector(question_3)
vector_4 = text_to_vector(question_4)

In [538]:
def question_classifier(question):
    question = text_to_vector(question.lower())
    cosine_similarity = np.zeros(4)
    cosine_similarity[0] = get_cosine(vector_1, question)
    cosine_similarity[1] = get_cosine(vector_2, question)
    cosine_similarity[2] = get_cosine(vector_3, question)
    cosine_similarity[3] = get_cosine(vector_4, question)
    
    max_index = np.argmax(cosine_similarity)
    
    return max_index

In [539]:
#2. extract entities

In [540]:
def extract_entities(sen):
    if type(sen) == str:
         NE = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sen)))
    else:
         NE = nltk.ne_chunk(nltk.pos_tag(sen))
    entities = []
    curr = []
    
    for i in NE:
        if type(i) == Tree:
            curr.append(" ".join([token for token, pos in i.leaves()]))
        elif curr:
            ne = " ".join(curr)
            if ne not in entities:
                entities.append(ne)
                curr =[]
        else:
            continue
    return entities

In [541]:
#2. define GDP question 

In [542]:
from nltk.corpus import stopwords
import pandas as pd

In [543]:
stop_words = set(stopwords.words('english'))

In [544]:
def GDP_question(question):
    
    token_question = nltk.word_tokenize(question)
    if '?' in token_question:
        token_question.remove('?')
    pos_question = nltk.pos_tag(token_question)
    
    keywords = []
    for i in range(len(token_question)-1,-1,-1):
        if pos_question[i][1] != 'TO' and pos_question[i][1] !='IN':
            keywords.insert(0,token_question[i])
        else:
            break
    ESquery = ['GDP','increase','decrease','growth','effect','effects','affect','affects','%','percent']
    for i in range(len(keywords)):
        ESquery.append(keywords[i])
    ESsearch = client.search(index = 'articles-index', q = ESquery, size =1000)
    EShits = ESsearch['hits']['hits']
    
    CandSen = []
    CandTok = []
    CandSco = []
    
    check = ESquery
    check.remove('GDP')
    for i in np.arange(len(EShits)):
        sentence = EShits[i]['_source']['body']
        token_sen = nltk.word_tokenize(sentence)
        score = 0
        if 'GDP' in token_sen:
            score += 2
        for i in np.arange(len(keywords)):
            if keywords[i] in token_sen:
                score += 2
        for i in np.arange(len(check)):
            if check[i] in token_sen:
                score += 1
        CandSen.append(sentence)
        
        CandTok.append(token_sen)
        CandSco.append(score)
    CanDF = pd.DataFrame(data = [CandSen, CandTok, CandSco], index = ['Sen','Token','Score'])
    CanDF = CanDF.transpose()
    CanDF = CanDF.sort_values(['Score'],ascending = False)
    CanDF.index = range(len(CanDF['Sen']))
    
    percents = ['%','percent','percentage']
    bestsen =[]
    bestscore = []
    bestanswers = []
    
    for i in range(3):
        bestsen.append(CanDF['Token'][i])
        score = np.zeros(len(bestsen[i]))
        for j in range(len(score)):
            GDPleft = 100
            GDPright = 100
            targLeft = 100
            targRight = 100
            is_percent = 0
            
            if bestsen[i][j] in percents:
                is_percent = 1
            count = 0
            for k in range(j, -1, -1):
                if bestsen[i][k] == 'GDP':
                    GDPleft == count
                    break
                else:
                    count += 1
            count = 0
            for k in range(j, -1, -1):
                if bestsen[i][k] == keywords[0]:
                    targLeft = count
                    break
                else:
                    count += 1
            count = 0
            for k in range(j, len(bestsen[i]), 1):
                if bestsen[i][k] == 'GDP':
                    GDPright = count
                    break
                else:
                    count += 1
            count = 0
            for k in range(j, len(bestsen[i]), 1):
                if bestsen[i][k] == keywords[0]:
                    targRight = count
                    break
                else:
                    count += 1
            GDPdist = min([GDPleft, GDPright])
            targdist = min([targLeft, targRight])
            score[j] = is_percent * 500 - GDPdist - targdist
            
        bestscore.append(score)
        index, value = max(enumerate(bestscore[i]), key = operator.itemgetter(1))
        answer = [bestsen[i][index-1], bestsen[i][index]]
        bestanswers.append(answer)
        
    for i in range(3):
        if bestanswers[i][1] in percents:
            bestanswers[i][1] = 'percent'
            output = bestanswers[0][0] + " "
            output += bestanswers[0][1]
            break
    return output

In [545]:
def Bankrupt_question(question):
    token_question = nltk.word_tokenize(question)
    token_question = [token for token in token_question if not token in stop_words]
    pos_question = nltk.pos_tag(token_question)
    
    ESquery = ['bankrupt','declared','filed','bankruptcy',]
    
    date = []
    for i in range(len(token_question)):
        pos = pos_question[i][1]
        
        if pos == 'NNP' or pos == 'CD':
            ESquery.append(pos_question[i][0])
            date.append(pos_question[i][0])
    ESquery.append
    ESsearch = client.search(index = 'articles-index', q = ESquery, size = 1000)
    CandSen = []
    CandTok = []
    EShits = ESsearch['hits']['hits']
    for i in np.arange(len(EShits)):
        sentence = EShits[i]['_source']['body']
        token_sentence = nltk.word_tokenize(sentence)
        token_sentence = [token for token in token_sentence if not token in stop_words]
        
        Date_inside = 1
        Bank_inside = 0
        Bank = ['bankruptcy','bankrupt']
        
        for i in range(len(date)):
            if date[i] not in token_sentence:
                Date_inside = 0
        for i in range(len(Bank)):
            if Bank[i] in token_sentence:
                Bank_inside = 1
        if Date_inside == 1 and Bank_inside ==1:
            CandSen.append(sentence)
            CandTok.append(token_sentence)
    candidates = []
    for i in np.arange(len(CandSen)):
        ne = extract_entities(CandSen[i])
        for j in np.arange(len(ne)):
            candidates.append(ne[j])
    if candidates != []:
        return max(set(candidates), key = candidates.count)
    else:
        print('no results')

In [546]:
def CEO_question(question):
    ne = extract_entities(question)
    ESsearch = client.search(index ='articles-index', q = ne, size = 1000)
    Candsen =[]
    for i in np.arange(len(ESsearch['hits']['hits'])):
        Candsen.append(ESsearch['hits']['hits'][i]['_source']['body'])
        Candsen[i] = nltk.word_tokenize(Candsen[i])
    tq = nltk.word_tokenize(ne[0])
    if 'of' in tq:
        tq.remove('of')
    if len(tq) == 1:
        companyname = tq[0]
    else:
        companyname = tq[1]
    if len(companyname) > 2:
        for i in range(2, len(tq)):
            companyname = companyname + ' ' + tq[i]
    rs =[]
    for i in np.arange(len(Candsen)):
        t = 1
        for j in np.arange(len(tq)):
            if tq[j] not in Candsen[i]:
                t = 0
        if t == 1:
            rs.append(Candsen[i])
    Candidates = []
    for i in np.arange(len(rs)):
        ce = extract_entities(rs[i])
        for j in np.arange(len(ce)):
            if (ce[j] != companyname) and (len(nltk.word_tokenize(ce[j]))) == 2:
                Candidates.append(ce[j])
    if Candidates != []:
        return max(set(Candidates), key = Candidates.count)
    else:
        print('No results')

In [547]:
def answer(question) :
    questiontype = question_classifier(question)
    
    if questiontype == 0 :
        return Bankrupt_question(question)
    elif questiontype == 1 :
        return("Consumption, consumer spending, government spending, investment, imports, exports, foreign trade")
    elif questiontype == 2 :
        return GDP_question(question)
    elif questiontype == 3 :
        return CEO_question(question) 

In [549]:
answer("What is the CEO of Twitter?")

'Jack Dorsey'

In [551]:
answer("What is the CEO of Microsoft?")

'Steve Ballmer'

In [552]:
answer("What is the CEO of Facebook?")

'Mark Zuckerberg'

In [554]:
answer("Which company went bankrupt in September 2008?")

'Lehman Brothers'

In [561]:
answer("Which company went bankrupt in July 2013?")

'Detroit'

In [572]:
answer("Which company went bankrupt in October 2014?")

'Reuters'

In [573]:
answer("What factors have the most effect on GDP?")

'Consumption, consumer spending, government spending, investment, imports, exports, foreign trade'

In [574]:
answer("what percentage is assoicated with foreign trade?")

'0.7 percent'

In [578]:
answer("what percentage change in GDP results from government spending?")

'.6 percent'